In [1]:
#import cv2
import numpy as np
from glob import glob
import os
from sklearn.svm import LinearSVC
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from PIL import Image
import PIL.ImageOps
from collections import defaultdict
from glob import glob
from random import shuffle, seed
import pylab as pl
import pandas as pd
import re
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import svm

In [2]:
def img_to_array(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = list(img.getdata())
    #img = map(list, img)
    img = np.array(img)
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

shirt = glob('bag/dataset/shirt_resized/*')
collared_tees = glob('bag/dataset/collared_tees_resized/*')
normal_tees = glob('bag/dataset/normal_tees_resized/*')

process_file = img_to_array

raw_data = [(process_file(filename),'shirt',filename) for filename in shirt] + \
           [(process_file(filename),'collared_tees',filename) for filename in collared_tees] + \
           [(process_file(filename),'normal_tees',filename) for filename in normal_tees]

# randomly order the data
seed(0)
shuffle(raw_data)

In [3]:
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])
y = [(0 if label == 'collared_tees' else( 1 if label=='normal_tees' else 2 )) for label in labels ]

# Scaling the words
stdSlr = StandardScaler().fit(data)
data = stdSlr.transform(data)

from sklearn.cross_validation import train_test_split,cross_val_score,KFold,StratifiedShuffleSplit,StratifiedKFold
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=2)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int32
  "got %s" % (estimator, X.dtype))


In [4]:
pca = PCA(n_components =55, whiten=True) ##55 - 63%
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=700, 
                           criterion='gini',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='sqrt',
                           bootstrap=False, 
                           random_state=40,
                           verbose=0) 
                           #min_density=None, 
                           #compute_importances=None)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred)) # accuracy = 0.86592178

0.524590163934


In [6]:
#from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
clf=RandomForestClassifier(n_estimators=700, 
                           criterion='gini',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='auto',
                           bootstrap=False, 
                           random_state=1,
                           verbose=0, 
                           min_density=None, 
                           compute_importances=None)

scores = cross_validation.cross_val_score(clf, data, y, cv=10)

In [8]:
scores

array([ 0.59677419,  0.4516129 ,  0.58064516,  0.48387097,  0.59016393,
        0.52459016,  0.58333333,  0.59322034,  0.62711864,  0.57627119])

In [32]:
img = cv2.imread("bag/test/thumbnails/stest.jpg",0)

#img = list(img.getdata())
img = map(list, img)
img = np.array(img)
s = img.shape[0] * img.shape[1]
img_wid = img.reshape(1, s)
img.shape

(128L, 128L)

In [33]:
stdSlr = StandardScaler().fit(img_wid[0])
img_wid1 = stdSlr.transform(img_wid[0])

In [34]:
img_wid1

array([ 1.01587,  0.96325, ..., -0.43121, -0.39175])

In [35]:
pca = PCA(n_components =55, whiten=True) ##55 - 63%
img_wid1=pca.fit_transform(img)

In [36]:
img_wid1.shape

(128L, 55L)

In [37]:
y_pred = clf.predict(img_wid1)
y_pred

array([0, 0, ..., 1, 1])

In [133]:
from sklearn.grid_search import GridSearchCV
e_range = (0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100,150,200)

param_grid = dict(random_state = e_range)                  
print param_grid

{'random_state': (0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 100, 150, 200)}


In [134]:
from sklearn.ensemble import RandomForestClassifier
'''
grid_clf = RandomForestClassifier(criterion='entropy',
                                  max_depth=None,
                                  min_samples_split=1,
                                  min_samples_leaf=11,
                                  max_features=0.5,
                                  n_jobs=-1, 
                                  random_state=42,
                                  min_density=None)
                                  '''
grid_clf = RandomForestClassifier(n_estimators=700, 
                           criterion='gini',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='auto',
                           bootstrap=False,n_jobs=-1, 
                           verbose=0, 
                           min_density=None, 
                           compute_importances=None)

In [135]:
grid =GridSearchCV(grid_clf,param_grid,cv=8,scoring ='accuracy')

In [136]:
grid.fit(data,y)

GridSearchCV(cv=8,
       estimator=RandomForestClassifier(bootstrap=False, compute_importances=None,
            criterion='gini', max_depth=7, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=700, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'random_state': (0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 100, 150, 200)},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [137]:
grid.grid_scores_

[mean: 0.56837, std: 0.05007, params: {'random_state': 0},
 mean: 0.55684, std: 0.04116, params: {'random_state': 5},
 mean: 0.55684, std: 0.04520, params: {'random_state': 10},
 mean: 0.56178, std: 0.05097, params: {'random_state': 15},
 mean: 0.56343, std: 0.04315, params: {'random_state': 20},
 mean: 0.55684, std: 0.03853, params: {'random_state': 25},
 mean: 0.54860, std: 0.03777, params: {'random_state': 30},
 mean: 0.57166, std: 0.05362, params: {'random_state': 35},
 mean: 0.54201, std: 0.03986, params: {'random_state': 40},
 mean: 0.55519, std: 0.05004, params: {'random_state': 45},
 mean: 0.56178, std: 0.05072, params: {'random_state': 50},
 mean: 0.56507, std: 0.05469, params: {'random_state': 55},
 mean: 0.56343, std: 0.04523, params: {'random_state': 60},
 mean: 0.56013, std: 0.04403, params: {'random_state': 65},
 mean: 0.55354, std: 0.04563, params: {'random_state': 70},
 mean: 0.55189, std: 0.04401, params: {'random_state': 75},
 mean: 0.56672, std: 0.04917, params: {'ra